In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### Data reading 

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@databricksetee.dfs.core.windows.net/products")

df.display()

In [0]:
df = df.drop("_rescued_data")

### **Functions**

In [0]:
df.createOrReplaceTempView("products")


In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
select product_id, price, databricks_cata.bronze.discount_func(price) as discounted_price
from products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
SELECT product_id, brand, databricks_cata.bronze.upper_func(brand) as brand_upper
FROM products

In [0]:
df.write.format("delta")\
    .option("overwriteSchema", "true")\
    .mode("overwrite")\
    .save("abfss://silver@databricksetee.dfs.core.windows.net/products")

In [0]:
df = spark.read.format("delta").load("abfss://silver@databricksetee.dfs.core.windows.net/products/")
display(df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databricksetee.dfs.core.windows.net/products/';